*IMPORTANDO PACOTES


In [ ]:
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import matplotlib.pyplot as plt

import datetime
import seaborn as sns
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


#Criando uma coluna OMS para identificar se o texto menciona OMS

In [ ]:
STF_STJ = pd.read_excel('/content/mydrive/MyDrive/OMS-Conteudos-STJ_STF.xlsx')
STF_STJ['OMS'] = np.nan
STF_STJ

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN
...,...,...,...,...,...,...,...,...
17415,2022-03-28,RHC,157460,13086443,stj,AgRg no RECURSO EM HABEAS CORPUS Nº 157.460 - ...,Tabaco_Tobacco,NaN
17416,2022-03-30,AREsp,2073755,12259496,stj,AGRAVO EM RECURSO ESPECIAL Nº 2.073.755 - PI (...,Tabaco_Tobacco,NaN
17417,2022-03-31,HC,727811,12545136,stj,AgRg no HABEAS CORPUS Nº 727.811 - RS (2022/00...,Tabaco_Tobacco,NaN
17418,2022-03-31,RHC,161755,13096384,stj,AgRg no RECURSO EM HABEAS CORPUS Nº 161.755 - ...,Tabaco_Tobacco,NaN


#Separando STF  E STJ

#STF

In [ ]:
STF = STF_STJ.loc[STF_STJ['Origem'] == 'stf']
STF

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN
...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN


#CRIANDO COLUNA TESTE PARA COMPARAR COM UM DATAFRAME COM A PALAVRA oms

In [ ]:
#criando coluna "teste_PROBABILIDADE"='PROBABILIDADE_OMS'   e   "teste_RESULTADO_df1 "='RESULTADO_OMS'
STF= STF.assign(PROBABILIDADE_OMS=0)
STF= STF.assign(RESULTADO_OMS=0)
STF

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0


#Criando coluna probabilidade e resualtado para fazer analise final

In [ ]:
#criando coluna "PROBABILIDADE"   e   "RESULTADO "
STF= STF.assign(PROBABILIDADE=0)
STF= STF.assign(RESULTADO=0)
STF

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN,0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN,0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN,0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN,0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0,0,0


#CRIANDO UM DATAFRAME COM OS TEXTOS OMS

In [ ]:
# Dados que serão usados para criar o DataFrame
dados = {'Nome': ['OMS', 'oms','Oms', 'Organização Mundial Saude','ORGANIZAÇÃO MUNDIAL SAUDE','WORLD HEALTH ORGANIZATION','World Health Organization'],
         }

# Criar o DataFrame
OMS = pd.DataFrame(dados)

# Mostrar o DataFrame
OMS

,Nome
0,OMS
1,oms
2,Oms
3,Organização Mundial Saude
4,ORGANIZAÇÃO MUNDIAL SAUDE
5,WORLD HEALTH ORGANIZATION
6,World Health Organization


#Fazendo a comparaçao com o dataframe criado com a palavra OMS

In [ ]:
print (len(OMS['Nome'][0]))
print (len(OMS['Nome'][1]))
print (len(OMS['Nome'][2]))
print (len(OMS['Nome'][3]))
print (len(OMS['Nome'][4]))
print (len(OMS['Nome'][5]))
print (len(OMS['Nome'][6]))

3
3
3
25
25
25
25


QUEM TEM A PALABRA OMS (SIM) QUEM NAO TEM (NAO)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
for i in range(len(STF)):
    texto_a_ser_comparado = STF['Conteudo'][i]
    P = []
    for j in range(len(OMS)):
        if len(OMS['Nome'][j]) ==3:
            texto_fonte = OMS['Nome'][j]  # Ajuste aqui para acessar a coluna 'Conteudo' em OMS
            n = 3
            counts = CountVectorizer(analyzer='word', ngram_range=(n, n))
            vocab2int = counts.fit([texto_a_ser_comparado, texto_fonte]).vocabulary_
            n_grams = counts.fit_transform([texto_a_ser_comparado, texto_fonte])
            n_grams_array = n_grams.toarray()
            intersection_list = np.amin(n_grams.toarray(), axis=0)
            intersection_count = np.sum(intersection_list)
            index_A = 0
            A_count = np.sum(n_grams.toarray()[index_A])
            p = intersection_count / A_count
            pt = round(p, 2)
            P.append(pt)
        elif len(OMS['Nome'][j]) ==25:
            texto_fonte = OMS['Nome'][j]  # Ajuste aqui para acessar a coluna 'Conteudo' em OMS
            n = 1
            counts = CountVectorizer(analyzer='word', ngram_range=(n, n))
            vocab2int = counts.fit([texto_a_ser_comparado, texto_fonte]).vocabulary_
            n_grams = counts.fit_transform([texto_a_ser_comparado, texto_fonte])
            n_grams_array = n_grams.toarray()
            intersection_list = np.amin(n_grams.toarray(), axis=0)
            intersection_count = np.sum(intersection_list)
            index_A = 0
            A_count = np.sum(n_grams.toarray()[index_A])
            p = intersection_count / A_count
            pt = round(p, 2)
            P.append(pt)
    probabilidade_maxima = max(P)
    localizacao = P.index(probabilidade_maxima)
    STF.loc[i, 'PROBABILIDADE_OMS'] = probabilidade_maxima
    STF.loc[i, 'RESULTADO_OMS'] = localizacao

In [ ]:
for i in range(len(STF)):
  if STF['PROBABILIDADE_OMS'][i]!=0:
    STF['OMS'][i]='sim'
  else:
    STF['OMS'][i]='não'

<ipython-input-29-e4c66469d9f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STF['OMS'][i]='não'


In [ ]:
## Filtrando o DataFrame com base em um valor específico da coluna 'Grupo'
grupo_especifico = 'sim'
GRUPO_SIM = STF[STF['OMS'] =='sim']
GRUPO_SIM_DESORDEM=GRUPO_SIM
GRUPO_SIM_DESORDEM

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
340,2020-04-01,Pet,8744,15342786248,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
341,2020-04-01,Pet,8749,15342785847,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
348,2020-04-02,HC,178600,15342789749,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
387,2020-04-02,HC,183376,15342797384,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
397,2020-04-03,ADPF,672,15342806401,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
493,2020-04-07,Pet,8755,15342832627,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
1129,2020-10-07,HC,192253,15344612072,stf,HABEAS CORPUS 192.253 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0
1130,2020-10-07,HC,192269,15344612068,stf,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,COVID-19,sim,0.01,3,0,0
1150,2020-10-13,HC,192540,15344651062,stf,HABEAS CORPUS 192.540 RIO DE JANEIRO\n\nRELATO...,COVID-19,sim,0.01,3,0,0
1190,2020-10-19,HC,192645,15344720238,stf,HABEAS CORPUS 192.645 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0


In [ ]:
len(GRUPO_SIM)

49

In [ ]:
# Filtrando ccom aqueles que não mencionam oms

GRUPO_NAO = STF[STF['OMS'] == 'não']
GRUPO_NAO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,não,0.0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,não,0.0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,não,0.0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,não,0.0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0


In [ ]:
len(GRUPO_NAO)

16474

#Ordenei o GRUPO_SIM

In [ ]:
GRUPO_SIM

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
340,2020-04-01,Pet,8744,15342786248,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
341,2020-04-01,Pet,8749,15342785847,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
348,2020-04-02,HC,178600,15342789749,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
387,2020-04-02,HC,183376,15342797384,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
397,2020-04-03,ADPF,672,15342806401,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
493,2020-04-07,Pet,8755,15342832627,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
1129,2020-10-07,HC,192253,15344612072,stf,HABEAS CORPUS 192.253 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0
1130,2020-10-07,HC,192269,15344612068,stf,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,COVID-19,sim,0.01,3,0,0
1150,2020-10-13,HC,192540,15344651062,stf,HABEAS CORPUS 192.540 RIO DE JANEIRO\n\nRELATO...,COVID-19,sim,0.01,3,0,0
1190,2020-10-19,HC,192645,15344720238,stf,HABEAS CORPUS 192.645 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0


In [ ]:
GRUPO_SIM.reset_index(drop=True, inplace=True)
GRUPO_SIM_ORDENADO=GRUPO_SIM
GRUPO_SIM_ORDENADO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2020-04-01,Pet,8744,15342786248,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
1,2020-04-01,Pet,8749,15342785847,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
2,2020-04-02,HC,178600,15342789749,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
3,2020-04-02,HC,183376,15342797384,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
4,2020-04-03,ADPF,672,15342806401,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
5,2020-04-07,Pet,8755,15342832627,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0
6,2020-10-07,HC,192253,15344612072,stf,HABEAS CORPUS 192.253 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0
7,2020-10-07,HC,192269,15344612068,stf,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,COVID-19,sim,0.01,3,0,0
8,2020-10-13,HC,192540,15344651062,stf,HABEAS CORPUS 192.540 RIO DE JANEIRO\n\nRELATO...,COVID-19,sim,0.01,3,0,0
9,2020-10-19,HC,192645,15344720238,stf,HABEAS CORPUS 192.645 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0


In [ ]:
GRUPO_NAO.reset_index(drop=True, inplace=True)
GRUPO_NAO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,não,0.0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,não,0.0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,não,0.0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,não,0.0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16469,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16470,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16471,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0
16472,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0


#NAO VOU FAZER EM GRUPOS . TUDO JUNTO

onde é grupos_1 , colocar GRUPOS_NAO

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
for i in range(len(GRUPO_NAO)):
  #o Grupo_NAO vai virar grupo_1 ate grupo_17
  texto_a_ser_comparado =GRUPO_NAO["Conteudo"][i]
  #texto_a_ser_comparado =GRUPO_NAO.iloc[i]['Conteudo']
  P=[]
  #for j in range(len(df1111)):
  for j in range(len(GRUPO_SIM)):
    #valor = GRUPO_SIM.iloc[0]['Conteudo']
    #texto_fonte=GRUPO_SIM.iloc[j]['Conteudo']
    texto_fonte= GRUPO_SIM["Conteudo"][j]
    n=1
    #instancia o contador de n-grams
    counts=CountVectorizer(analyzer='word',ngram_range=(n,n))
    #cria um dicionario de n grams
    vocab2int=counts.fit([texto_a_ser_comparado,texto_fonte]).vocabulary_
    n_grams=counts.fit_transform([texto_a_ser_comparado,texto_fonte])
    #cria uma matriz de contagem para os dois textos
    n_grams_array = n_grams.toarray()
    #matriz de

    intersection_list= np.amin(n_grams.toarray(),axis=0)
    #intersection_list= np.amin(n_grams.toarray())
    intersection_list
   #contagem interseçao
    intersection_count= np.sum(intersection_list)
    index_A=0
    A_count=np.sum(n_grams.toarray()[index_A])
    p=intersection_count/A_count
    pt = round(p, 2)
    P.append(pt)
    #colocar o vetor e adicionar este p no vetor
  B=[]
  Z=[]
  probabilidade_maxima = max(P)
  localizaçao  = P.index(probabilidade_maxima)
  GRUPO_NAO['PROBABILIDADE'][i]=probabilidade_maxima
  GRUPO_NAO['RESULTADO'][i]=localizaçao


A saída de streaming foi truncada nas últimas 5000 linhas.
<ipython-input-37-3443e24dfcf4>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-37-3443e24dfcf4>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['RESULTADO'][i]=localizaçao
<ipython-input-37-3443e24dfcf4>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['PROBABILIDADE'][i]=probabilidade_m

In [ ]:
GRUPO_NAO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,não,0.0,0,0.08,3
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,não,0.0,0,0.04,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,não,0.0,0,0.19,7
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,não,0.0,0,0.05,7
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,não,0.0,0,0.08,7
...,...,...,...,...,...,...,...,...,...,...,...,...
16469,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0.08,3
16470,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0.08,3
16471,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0.05,7
16472,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0.06,7


#Trabalhando com datas

In [ ]:
GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
GRUPO_NAO['Data'] = pd.to_datetime(GRUPO_NAO['Data'])

<ipython-input-39-d8d3c2b9ad28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
<ipython-input-39-d8d3c2b9ad28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Data'] = pd.to_datetime(GRUPO_NAO['Data'])


#considerando quem tem probabilidade maior que 0.7

In [ ]:
condicao = GRUPO_NAO['PROBABILIDADE'] > 0.7

# Filtrar o DataFrame com base na condição
GRUPO_NAO = GRUPO_NAO[condicao]

# Exibir o DataFrame filtrado
GRUPO_NAO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
299,2020-03-31,Pet,8740,15342768496,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.76,1
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4
530,2020-04-14,HC,183522,15342849504,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7
1664,2020-12-01,HC,194683,15345119863,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,29
5894,2021-07-07,MS,38036,15347001079,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.036...,COVID-19,não,0.0,0,0.93,45
5895,2021-07-07,MS,38038,15347001082,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.038...,COVID-19,não,0.0,0,0.89,45
5896,2021-07-07,MS,38039,15347001083,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.039...,COVID-19,não,0.0,0,0.90,45


In [ ]:
#criando uma coluna repetindo o indice neste dataframe GRUPO_SIM
GRUPO_SIM['indice'] = GRUPO_SIM.index
GRUPO_SIM

<ipython-input-41-f7d753fe9fe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['indice'] = GRUPO_SIM.index


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,indice
0,2020-04-01,Pet,8744,15342786248,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0,0
1,2020-04-01,Pet,8749,15342785847,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0,1
2,2020-04-02,HC,178600,15342789749,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0,2
3,2020-04-02,HC,183376,15342797384,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0,3
4,2020-04-03,ADPF,672,15342806401,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0,4
5,2020-04-07,Pet,8755,15342832627,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,sim,0.01,3,0,0,5
6,2020-10-07,HC,192253,15344612072,stf,HABEAS CORPUS 192.253 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0,6
7,2020-10-07,HC,192269,15344612068,stf,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,COVID-19,sim,0.01,3,0,0,7
8,2020-10-13,HC,192540,15344651062,stf,HABEAS CORPUS 192.540 RIO DE JANEIRO\n\nRELATO...,COVID-19,sim,0.01,3,0,0,8
9,2020-10-19,HC,192645,15344720238,stf,HABEAS CORPUS 192.645 PARANÁ\n\nRELATOR : MIN....,COVID-19,sim,0.01,3,0,0,9


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
GRUPO_NAO['Data_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])
GRUPO_NAO

<ipython-input-42-5fc938796263>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Data_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM
299,2020-03-31,Pet,8740,15342768496,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.76,1,2020-04-01
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4,2020-04-03
530,2020-04-14,HC,183522,15342849504,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07
1664,2020-12-01,HC,194683,15345119863,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,29,2021-04-06
5894,2021-07-07,MS,38036,15347001079,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.036...,COVID-19,não,0.0,0,0.93,45,2021-07-07
5895,2021-07-07,MS,38038,15347001082,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.038...,COVID-19,não,0.0,0,0.89,45,2021-07-07
5896,2021-07-07,MS,38039,15347001083,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.039...,COVID-19,não,0.0,0,0.90,45,2021-07-07


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
GRUPO_NAO['Major Health Topics_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Major Health Topics'])
GRUPO_NAO

<ipython-input-43-aa51a581b2b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Major Health Topics_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Major Health Topics'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Major Health Topics_SIM
299,2020-03-31,Pet,8740,15342768496,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.76,1,2020-04-01,COVID-19
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4,2020-04-03,COVID-19
530,2020-04-14,HC,183522,15342849504,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19
1664,2020-12-01,HC,194683,15345119863,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,29,2021-04-06,COVID-19
5894,2021-07-07,MS,38036,15347001079,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.036...,COVID-19,não,0.0,0,0.93,45,2021-07-07,COVID-19
5895,2021-07-07,MS,38038,15347001082,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.038...,COVID-19,não,0.0,0,0.89,45,2021-07-07,COVID-19
5896,2021-07-07,MS,38039,15347001083,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.039...,COVID-19,não,0.0,0,0.90,45,2021-07-07,COVID-19


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
GRUPO_NAO['Conteudo_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Conteudo'])
GRUPO_NAO

<ipython-input-44-7342044935f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Conteudo_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Conteudo'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Major Health Topics_SIM,Conteudo_SIM
299,2020-03-31,Pet,8740,15342768496,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.76,1,2020-04-01,COVID-19,Documento assinado digitalmente conforme MP n°...
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4,2020-04-03,COVID-19,Documento assinado digitalmente conforme MP n°...
530,2020-04-14,HC,183522,15342849504,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1664,2020-12-01,HC,194683,15345119863,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,29,2021-04-06,COVID-19,HABEAS CORPUS 199.702 SÃO PAULO\n\nRELATOR : M...
5894,2021-07-07,MS,38036,15347001079,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.036...,COVID-19,não,0.0,0,0.93,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
5895,2021-07-07,MS,38038,15347001082,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.038...,COVID-19,não,0.0,0,0.89,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
5896,2021-07-07,MS,38039,15347001083,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.039...,COVID-19,não,0.0,0,0.90,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
GRUPO_NAO['Data_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])
GRUPO_NAO

<ipython-input-45-5fc938796263>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Data_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Major Health Topics_SIM,Conteudo_SIM
299,2020-03-31,Pet,8740,15342768496,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.76,1,2020-04-01,COVID-19,Documento assinado digitalmente conforme MP n°...
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4,2020-04-03,COVID-19,Documento assinado digitalmente conforme MP n°...
530,2020-04-14,HC,183522,15342849504,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1664,2020-12-01,HC,194683,15345119863,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,29,2021-04-06,COVID-19,HABEAS CORPUS 199.702 SÃO PAULO\n\nRELATOR : M...
5894,2021-07-07,MS,38036,15347001079,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.036...,COVID-19,não,0.0,0,0.93,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
5895,2021-07-07,MS,38038,15347001082,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.038...,COVID-19,não,0.0,0,0.89,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
5896,2021-07-07,MS,38039,15347001083,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.039...,COVID-19,não,0.0,0,0.90,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...


##CONSIDERAMOS AGORA AS DATAS

Para que o GRUPO_SIM tenha influenciado o grupo_1 , precisamos verificar que o GRUPO_SIM tenha uma data mais antiga que o grupo_1

In [ ]:
GRUPO_NAO['Data'] = pd.to_datetime(GRUPO_NAO['Data'])
GRUPO_NAO['Data_SIM'] = pd.to_datetime(GRUPO_NAO['Data_SIM'])
GRUPO_NAO = GRUPO_NAO[GRUPO_NAO['Data'] > GRUPO_NAO['Data_SIM']]
GRUPO_NAO


<ipython-input-46-26a983dc9108>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Data'] = pd.to_datetime(GRUPO_NAO['Data'])
<ipython-input-46-26a983dc9108>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['Data_SIM'] = pd.to_datetime(GRUPO_NAO['Data_SIM'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Major Health Topics_SIM,Conteudo_SIM
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4,2020-04-03,COVID-19,Documento assinado digitalmente conforme MP n°...
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...
5942,2021-07-12,MS,38042,15347038096,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.042...,COVID-19,não,0.0,0,0.79,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
6033,2021-07-20,MS,38070,15347088175,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.070...,COVID-19,não,0.0,0,0.75,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
6647,2021-08-27,MS,38187,15347487911,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.187...,COVID-19,não,0.0,0,0.74,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...
11531,2016-05-09,ARE,733577,309396844,stf,RECURSO EXTRAORDINÁRIO COM AGRAVO 733.577 DIST...,Governança de Sáude Pública_Health system gove...,não,0.0,0,0.94,48,2016-05-05,Governança de Sáude Pública_Health system gove...,RECURSO EXTRAORDINÁRIO COM AGRAVO 749.016 RIO ...


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
GRUPO_NAO['tipo_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])
GRUPO_NAO

<ipython-input-47-3bcdf95fb825>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_NAO['tipo_SIM'] =GRUPO_NAO['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Major Health Topics_SIM,Conteudo_SIM,tipo_SIM
491,2020-04-13,Ext,1614,15342829868,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.71,4,2020-04-03,COVID-19,Documento assinado digitalmente conforme MP n°...,ADPF
1307,2020-10-28,HC,193322,15344810163,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.80,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,HC
1317,2020-10-28,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,HC
1339,2020-10-29,RHC,192375,15344822841,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0.75,7,2020-10-07,COVID-19,HABEAS CORPUS 192.269 SÃO PAULO\n\nRELATOR : M...,HC
5942,2021-07-12,MS,38042,15347038096,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.042...,COVID-19,não,0.0,0,0.79,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...,MS
6033,2021-07-20,MS,38070,15347088175,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.070...,COVID-19,não,0.0,0,0.75,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...,MS
6647,2021-08-27,MS,38187,15347487911,stf,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.187...,COVID-19,não,0.0,0,0.74,45,2021-07-07,COVID-19,MEDIDA CAUTELAR EM MANDADO DE SEGURANÇA 38.040...,MS
11531,2016-05-09,ARE,733577,309396844,stf,RECURSO EXTRAORDINÁRIO COM AGRAVO 733.577 DIST...,Governança de Sáude Pública_Health system gove...,não,0.0,0,0.94,48,2016-05-05,Governança de Sáude Pública_Health system gove...,RECURSO EXTRAORDINÁRIO COM AGRAVO 749.016 RIO ...,ARE


**Agora ja se pode fazera contagem para as redes**

In [ ]:
nome_do_arquivo = 'GRUPO_NAO_novo.xlsx'
GRUPO_NAO.to_excel(nome_do_arquivo, index=False)

# Local onde o arquivo é salvo no Google Colab
print("O arquivo foi salvo em:", nome_do_arquivo)

O arquivo foi salvo em: GRUPO_NAO_novo.xlsx


In [ ]:
nome_do_arquivo = 'GRUPO_NAO.xlsx'
GRUPO_NAO.to_excel(nome_do_arquivo, index=False)

# Local onde o arquivo é salvo no Google Colab
print("O arquivo foi salvo em:", nome_do_arquivo)

O arquivo foi salvo em: GRUPO_NAO.xlsx


In [ ]:
from google.colab import drive
import shutil

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para o arquivo Excel no sistema de arquivos temporário do Google Colab
caminho_arquivo_colab = 'GRUPO_NAO.xlsx'

# Definir o caminho para onde queremos mover o arquivo no Google Drive
caminho_destino_drive = '/content/drive/MyDrive/GRUPO_NAO.xlsx'

# Mover o arquivo para o Google Drive
shutil.move(caminho_arquivo_colab, caminho_destino_drive)

print("Arquivo movido com sucesso para o Google Drive em:", caminho_destino_drive)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo movido com sucesso para o Google Drive em: /content/drive/MyDrive/GRUPO_NAO.xlsx


In [ ]:
from google.colab import drive
import shutil

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para o arquivo Excel no sistema de arquivos temporário do Google Colab
caminho_arquivo_colab = 'GRUPO_NAO_novo.xlsx'

# Definir o caminho para onde queremos mover o arquivo no Google Drive
caminho_destino_drive = '/content/drive/MyDrive/GRUPO_NAO_novo.xlsx'

# Mover o arquivo para o Google Drive
shutil.move(caminho_arquivo_colab, caminho_destino_drive)

print("Arquivo movido com sucesso para o Google Drive em:", caminho_destino_drive)

Mounted at /content/drive
Arquivo movido com sucesso para o Google Drive em: /content/drive/MyDrive/GRUPO_NAO_novo.xlsx
